In [1]:
import requests
import re
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
import time
import os
import json

In [16]:
user_agent = {'User-agent': 'Mozilla/5.0'}

chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

driver = webdriver.Chrome(chromedriver)

In [3]:
def get_film_data(film_id):
    
    delimiter = '+_+'
    
    film_url = "http://www.imdb.com/title/" + film_id
    driver.get(film_url)
    
    response = requests.get(film_url, headers = user_agent)
    html = response.text
    soup = bs(html, 'lxml')
    
    film_dict = {}
    
    try:
        genre_selector = "//*[@id='titleStoryLine']//a[contains(@href, '/genre/')]"
        film_dict['genres'] = ''
        for genre in driver.find_elements_by_xpath(genre_selector):
            film_dict['genres'] = film_dict['genres'] + delimiter + genre.text
    except NoSuchElementException:
        film_dict['genres'] = None
        
    try:
        title_selector = "//*[@id='title-overview-widget']//*[@itemprop='name']"
        film_dict['title'] = driver.find_element_by_xpath(title_selector).text
    except NoSuchElementException:
        film_dict['title'] = None
    
    try:
        year_selector = '//*[@id="titleYear"]'
        film_dict['year'] = driver.find_element_by_xpath(year_selector).text.split('(')[1].split(')')[0]
    except NoSuchElementException:
        film_dict['year'] = None  
    
    try:
        rating_selector = "//*[@id='title-overview-widget']//*[@itemprop='ratingValue']"
        film_dict['rating'] = driver.find_element_by_xpath(rating_selector).text
    except NoSuchElementException:
        film_dict['rating'] = None  
   
    try:
        votes_selector = "//*[@id='title-overview-widget']//*[@itemprop='ratingCount']"
        film_dict['votes'] = driver.find_element_by_xpath(votes_selector).text
    except NoSuchElementException:
        film_dict['votes'] = None     
    
    try:
        directors_selector = "//*[@id='title-overview-widget']//*[@itemprop='director']"
        film_dict['directors'] = ''
        for director in driver.find_elements_by_xpath(directors_selector):
            film_dict['directors'] = film_dict['directors'] + delimiter + director.text.split('(')[0].strip()
    except NoSuchElementException:
        film_dict['directors'] = None    
  
    try:
        writers_selector = "//*[@id='title-overview-widget']//*[@itemprop='creator']"
        film_dict['writers'] = ''
        for writer in driver.find_elements_by_xpath(writers_selector):
            film_dict['writers'] = film_dict['writers'] + delimiter + writer.text.split('(')[0].strip()
    except NoSuchElementException:
        film_dict['writers'] = None     

    try:
        actors_selector = "//*[@id='titleCast']//*[@itemprop='name']"
        film_dict['actors'] = ''
        for actor in driver.find_elements_by_xpath(actors_selector):
            film_dict['actors'] = film_dict['actors'] + delimiter + actor.text
    except NoSuchElementException:
        film_dict['actors'] = None          

    try:
        keywords_selector = "//*[@class='see-more inline canwrap']//*[@itemprop='keywords']"
        film_dict['keywords'] = ''
        for keyword in driver.find_elements_by_xpath(keywords_selector):
            film_dict['keywords'] = film_dict['keywords'] + delimiter + keyword.text
    except NoSuchElementException:
        film_dict['keywords'] = None       

    try:
        mpaa_selector = "//meta[@itemprop='contentRating']"
        film_dict['mpaa'] = driver.find_element_by_xpath(mpaa_selector).get_attribute('content')
    except NoSuchElementException:
        film_dict['mpaa'] = None      

    try:
        release_date_selector = "//meta[@itemprop='datePublished']"
        film_dict['release_date'] = driver.find_element_by_xpath(release_date_selector).get_attribute('content')
    except NoSuchElementException:
        film_dict['release_date'] = None     

    try:
        languages_selector = "//*[@id='titleDetails']//a[contains(@href, 'language')]"
        film_dict['languages'] = ''
        for language in driver.find_elements_by_xpath(languages_selector):
            film_dict['languages'] = film_dict['languages'] + delimiter + language.text
    except NoSuchElementException:
        film_dict['languages'] = None   
    
    try:
        countries_selector = "//*[@id='titleDetails']//a[contains(@href, 'country_of_origin')]"
        film_dict['countries'] = ''
        for country in driver.find_elements_by_xpath(countries_selector):
            film_dict['countries'] = film_dict['countries'] + delimiter + country.text
    except NoSuchElementException:
        film_dict['countries'] = None 
    
    try:
        runtime_selector = "//*[@id='titleDetails']//*[@itemprop='duration']"
        film_dict['runtime'] = int(driver.find_element_by_xpath(runtime_selector).text.split(' ')[0])
    except NoSuchElementException:
        film_dict['runtime'] = None 
     
    film_dict['budget'] = None
    film_dict['gross_usa'] = None
    for h4 in soup.find_all('h4'):
        if "Budget:" in h4:
            film_dict['budget'] = h4.next_sibling        
        if "Gross USA:" in h4:
            film_dict['gross_usa'] = h4.next_sibling
            
    return film_dict

In [4]:
with open("film_ids.json","r") as f:
      data = f.read()
        
film_ids = json.loads(data)

In [5]:
dict_list = []
dummy = 0

for film_id in film_ids[47000:48000]:
    print(dummy)
    dict_list.append(get_film_data(film_id))
    dummy = dummy + 1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214


TimeoutException: Message: timeout
  (Session info: chrome=63.0.3239.132)
  (Driver info: chromedriver=2.35.528157 (4429ca2590d6988c0745c24c8858745aaaec01ef),platform=Mac OS X 10.13.2 x86_64)


In [6]:
film_ids[47214]

'tt3501632'

In [11]:
dummy = 214

for film_id in film_ids[47214:48000]:
    print(dummy)
    dict_list.append(get_film_data(film_id))
    dummy = dummy + 1

214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
375
376
377
378
379
380
381
382
383
384
385
386
387
388
389
390
391
392
393
394
395
396
397
398
399
400
401
402
403
404
405
406
407
408
409
410
411
412
413
414
415
416
417
418
419
420
421
422
423
424
425
426
427
428
429
430
431
432
433
434
435
436
437
438
439
440
441
442
443
444
445
446
447
448
449
450
451
452
453
454
455
456
457
458
459
460
461
462
463


TimeoutException: Message: timeout
  (Session info: chrome=63.0.3239.132)
  (Driver info: chromedriver=2.35.528157 (4429ca2590d6988c0745c24c8858745aaaec01ef),platform=Mac OS X 10.13.2 x86_64)


In [12]:
dummy = 734

for film_id in film_ids[47734:48000]:
    print(dummy)
    dict_list.append(get_film_data(film_id))
    dummy = dummy + 1

734
735
736
737
738
739
740
741
742
743
744
745
746
747
748
749
750
751
752
753
754
755
756
757
758
759
760
761
762
763
764
765


TimeoutException: Message: timeout
  (Session info: chrome=63.0.3239.132)
  (Driver info: chromedriver=2.35.528157 (4429ca2590d6988c0745c24c8858745aaaec01ef),platform=Mac OS X 10.13.2 x86_64)


In [17]:
dummy = 766

for film_id in film_ids[47766:48000]:
    print(dummy)
    dict_list.append(get_film_data(film_id))
    dummy = dummy + 1

766
767
768
769
770
771
772
773
774
775
776
777
778
779
780
781
782
783
784
785
786
787
788
789
790
791
792
793
794
795
796
797
798
799
800
801
802
803
804
805
806
807
808
809
810
811
812
813
814
815
816
817
818
819
820
821
822
823
824
825
826
827
828
829
830
831
832
833
834
835
836
837
838
839
840
841
842
843
844
845
846
847
848
849
850
851
852
853
854
855
856
857
858
859
860
861
862
863
864
865
866
867
868
869
870
871
872
873
874
875
876
877
878
879
880
881
882
883
884
885
886
887
888
889
890
891
892
893
894
895
896
897
898
899
900
901
902
903
904
905
906
907
908
909
910
911
912
913
914
915
916
917
918
919
920
921
922
923
924
925
926
927
928
929
930
931
932
933
934
935
936
937
938
939
940
941
942
943
944
945
946
947
948
949
950
951
952
953
954
955
956
957
958
959
960
961
962
963
964
965
966
967
968
969
970
971
972
973
974
975
976
977
978
979
980
981
982
983
984
985
986
987
988
989
990
991
992
993
994
995
996
997
998
999


In [19]:
#765 - The Last Jedi

In [20]:
The_Last_Jedi = {'actors': "+_+Mark Hamill+_+Carrie Fisher+_+Adam Driver+_+Daisy Ridley+_+John Boyega+_+Oscar Isaac+_+Andy Serkis+_+Lupita Nyong'o+_+Domhnall Gleeson+_+Anthony Daniels+_+Gwendoline Christie+_+Kelly Marie Tran+_+Laura Dern+_+Benicio Del Toro+_+Frank Oz",
 'budget': None,
 'countries': '+_+USA', 
 'directors': '+_+Rian Johnson',
 'genres': '+_+Action+_+Adventure+_+Fantasy',
 'gross_usa': ' $607,508,821, ',
 'keywords': '+_+wisecrack humor+_+deception+_+betrayal+_+hatred+_+torture',
 'languages': '+_+English',
 'mpaa': 'PG-13',
 'rating': '7.5',
 'release_date': '2017-12-15',
 'runtime': 152,
 'title': 'Star Wars: The Last Jedi (2017)',
 'votes': '302,136',
 'writers': "+_+Rian Johnson+_+George Lucas",  
 'year': '2017'}

In [21]:
The_Last_Jedi

{'actors': "+_+Mark Hamill+_+Carrie Fisher+_+Adam Driver+_+Daisy Ridley+_+John Boyega+_+Oscar Isaac+_+Andy Serkis+_+Lupita Nyong'o+_+Domhnall Gleeson+_+Anthony Daniels+_+Gwendoline Christie+_+Kelly Marie Tran+_+Laura Dern+_+Benicio Del Toro+_+Frank Oz",
 'budget': None,
 'countries': '+_+USA',
 'directors': '+_+Rian Johnson',
 'genres': '+_+Action+_+Adventure+_+Fantasy',
 'gross_usa': ' $607,508,821, ',
 'keywords': '+_+wisecrack humor+_+deception+_+betrayal+_+hatred+_+torture',
 'languages': '+_+English',
 'mpaa': 'PG-13',
 'rating': '7.5',
 'release_date': '2017-12-15',
 'runtime': 152,
 'title': 'Star Wars: The Last Jedi (2017)',
 'votes': '302,136',
 'writers': '+_+Rian Johnson+_+George Lucas',
 'year': '2017'}

In [22]:
dict_list.append(The_Last_Jedi)

In [23]:
len(dict_list)

1000

In [24]:
import pandas as pd

data = json.dumps(dict_list)

with open("film_data_48.json","w") as f:
      f.write(data)

with open("film_data_48.json","r") as f:
      data = f.read()
        
d = json.loads(data)

df = pd.DataFrame(d)

df

,actors,budget,countries,directors,genres,gross_usa,keywords,languages,mpaa,rating,release_date,runtime,title,votes,writers,year
0,+_+Tasha Tacosa+_+Robert Rhine+_+Brenton Jones...,None,+_+USA,+_+Jeff Leroy,+_+Sci-Fi,None,,+_+English,None,2.4,2017-09-18,85.0,Aliens vs. College Girls (2017),83,+_+Ted Chalmers+_+Keith Parker,2017
1,+_+Ajibike Adekoya+_+De'aundre Bonds+_+Cleave ...,None,+_+USA+_+Belize,+_+Jah,+_+Action+_+Drama+_+Sport+_+Thriller,None,,+_+English,PG-13,None,2017-09-18,108.0,Hogan (2017),None,+_+Jah+_+Jah,2017
2,+_+Jessica Jade Andres+_+Jessica Clark+_+Gerar...,"$15,000\n",+_+USA,+_+Carlton Byrd,+_+Comedy+_+Drama,None,+_+dark comedy+_+pop culture+_+sports arena,+_+English,None,None,2017-09-18,NaN,Meat Puppet: The Filmed Experience (2017),None,+_+Christopher Gabriel Núñez,2017
3,,None,+_+USA,,+_+Horror,None,,+_+English,None,8.6,2017-09-18,NaN,Halloween: The Return of Laurie Strode (2017),12,+_+Omar Azimi,2017
4,+_+Becka Adams+_+Anthony Backman+_+Elena Chern...,None,+_+USA,"+_+Chris Barnes,+_+Alessandro Frosali",+_+Horror,None,+_+two word title,+_+English,None,4.1,2017-09-19,78.0,Strange Events (2017),53,"+_+Nick Adamski-O'Toole,+_+Jay Holben",2017
5,+_+Steven Michael Quezada+_+Rachel G. Whittle+...,None,+_+USA,+_+Brett Bentman,+_+Horror,None,+_+halloween,+_+English,None,5.3,2017-09-19,81.0,The Night Before (2017),108,+_+Brett Bentman,2017
6,+_+Wade Sullivan+_+Cynthia Calvert+_+Felissa R...,None,+_+USA,+_+Steve Taylor,+_+Horror,None,+_+creature feature+_+creature+_+monster,+_+English,None,5.5,2017-09-19,95.0,Clawed (2017),37,"+_+Steve Taylor,+_+Rahul Chatterjee",2017
7,+_+Thomas Burr+_+Evans Forde+_+Eve Gordon+_+Ma...,None,+_+USA,+_+J.M. Logan,+_+Comedy,None,,+_+English,None,None,2017-09-19,NaN,The Garage Sale (2017),None,+_+Molly Hagan,2017
8,+_+Lily Alejandra+_+Sergio Briones+_+Brandon C...,$500\n,+_+USA,,+_+Horror,None,,+_+English,None,None,2017-09-19,NaN,Possessed (2017),None,+_+Sergio Briones,2017
9,+_+Lynnette Arthur+_+Jody Lee Bradley+_+Pat Ca...,"$200,000\n",+_+USA,+_+Jahida Diaab,+_+Comedy,None,+_+sexual+_+dysfunctional love+_+wiener+_+caes...,+_+English,None,None,2017-09-19,NaN,Short Comings (2017),None,+_+Jahida Diaab,2017
